# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MVP-Code"
finetune_tags = ["smol-course", "module_1"]

# Generate with the base model

Here we will try out the base model which does not have a chat template. 

In [3]:
# Let's test the base model before training
prompt = "write a python code implementing linked list"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
write a python code implementing linked list
assistant
write a python code implementing linked list

## 1.1.1.1

## 1.1.1.2

## 1.1.1.3

## 1.1.1.4

## 1.1.1.5

## 1.1.1.6

## 1.1.1.7

## 1.1.1.8

## 1


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [4]:
# Load a sample dataset

from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
# ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

ds_org = load_dataset("iamtarun/python_code_instructions_18k_alpaca")

In [5]:
ds = ds_org['train'].train_test_split(test_size=0.2,seed=442)

In [6]:
a=ds['train'].select(range(100))

In [7]:
l = 10
data = a[15:15+l]
for i in range(l):
    for k,v in data.items():
        if "prompt" in k: continue
        print(f"{k}: {v[i]}",'\n*')
    
    print('*'*5)
    

instruction: Write a Python program to decode the given encoded text using ROT-13 algorithm. 
*
input: Gur pynff vf snpgbevnag 
*
output: The code is obfuscated 
*
*****
instruction: Compose a program in Python to retrieve the most frequent letter in a given string. 
*
input: String: "Programmers" 
*
output: def most_frequent(input_string):
    # Convert input string to a dictionary
    char_frequency = {}
    for n in input_string:
        keys = char_frequency.keys()
        if n in keys:
            char_frequency[n] += 1
        else:
            char_frequency[n] = 1
     
    # Sort the dictionary by value in descending order
    max_value = max(char_frequency.values())
    max_char = [element for element in char_frequency if char_frequency[element] == max_value]
 
    # Print the most frequent letter
    if len(max_char) > 1:
        print("Most frequent character: " + ",".join(max_char))
    else:
        print("Most frequent character: " + max_char[0])

most_frequent("Programm

In [8]:
def process_sample(sample):
    messages=[]
    for k,v in sample.items():
        if not v: continue
        if k=="instruction" or k=="input":
            messages.append({"role": "user", "content":v})
        elif k=="output":
            messages.append({"role": "assistant", "content":v})
    sample["text"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    return sample

In [9]:
ds2 = ds.map(process_sample)

In [10]:
ds2

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'text'],
        num_rows: 14889
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'text'],
        num_rows: 3723
    })
})

In [11]:
b=ds2['train'].select(range(100))
b[:3]['text']

['<|im_start|>user\nCreate a Python program to compare two strings to determine the number of characters which are different.<|im_end|>\n<|im_start|>user\nNot applicable<|im_end|>\n<|im_start|>assistant\ndef compareStrings(str1, str2):\n count = 0\n \n for i in range(min(len(str1), len(str2))):\n if str1[i] != str2[i]:\n count += 1\n \n # if the strings are different lengths, add the remaining characters from the longer string to the count\n if len(str1) > len(str2):\n count += len(str1) - len(str2)\n elif len(str2) > len(str1):\n count += len(str2) - len(str1)\n \n return count<|im_end|>\n',
 '<|im_start|>user\nCreate an AI model in Python to classify emails from the Enron Spam Dataset as either spam or non-spam.<|im_end|>\n<|im_start|>user\nNot applicable<|im_end|>\n<|im_start|>assistant\nimport pandas as pd\nfrom sklearn.feature_extraction.text import CountVectorizer\nfrom sklearn.model_selection import train_test_split\nfrom sklearn import svm\nfrom sklearn.metrics import precision

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [12]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    dataset_text_field="text",
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=2,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id="sft-python",  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds2["train"],
    tokenizer=tokenizer,
    eval_dataset=ds2["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column

/data/nvidia_local/opensource/smol-course/.venv/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/data/nvidia_local/opensource/smol-course/.venv/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [13]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

Step,Training Loss,Validation Loss
50,0.897800,0.987215
100,1.180900,0.972966
150,0.805300,0.955371
200,0.733300,0.947304
250,0.902500,0.940802
300,0.796200,0.937213
350,1.011900,0.933743
400,0.896300,0.929066
450,0.737300,0.926517
500,0.986300,0.922398


In [15]:
trainer.push_to_hub(tags=finetune_tags)

training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/PavanMV/sft-python/commit/6181c726e50516c0083de7633a79b1347540ba7d', commit_message='End of training', commit_description='', oid='6181c726e50516c0083de7633a79b1347540ba7d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/PavanMV/sft-python', endpoint='https://huggingface.co', repo_type='model', repo_id='PavanMV/sft-python'), pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [14]:
prompt = "write a python code implementing linked list "
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("After training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

After training:
user
write a python code implementing linked list 
assistant
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def append(self, data):
        new_node = Node(data)
        if self.head is None:
            self.head = new_node
            return
        prev = self.head
        while prev.next:
            prev = prev.next
        prev.next


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.